In [ ]:
import urllib.parse as urlparse

import ipywidgets as ipw

In [ ]:
import sys

from aiidalab.__main__ import _find_version_to_install, _parse_requirement


def identify_app_and_version(app_query, prereleases, force):
    try:
        requirement = _parse_requirement(app_query)
    except Exception as error:
        raise RuntimeError(str(error)) from error

    try:
        return _find_version_to_install(
            requirement,
            dependencies="install",
            force=force,
            python_bin=sys.executable,
            prereleases=prereleases,
        )
    except Exception as error:
        raise RuntimeError(str(error)) from error

In [ ]:
url = urlparse.urlsplit(jupyter_notebook_url)

query = urlparse.parse_qs(url.query)

In [ ]:
from IPython.display import Javascript

from home.widgets import LogOutputWidget, Spinner

header = ipw.HTML()
logs = LogOutputWidget(layout=ipw.Layout(min_height="250px", max_height="100px"))
spinner = Spinner()

display(ipw.VBox([ipw.HBox([header, spinner]), logs]))

try:
    try:
        app_query = query["app"][0]
    except KeyError:
        raise RuntimeError("No app to install specified.")

    force = query.get("force", [False])[0]
    prereleases = query.get("prereleases", [False])[0]

    app, version = identify_app_and_version(
        query["app"][0], force=force, prereleases=prereleases
    )
except Exception as error:
    header.value = f"Error: {error}"

try:
    if version is None:
        header.value = "App already installed."
    else:
        header.value = f"Installing app '{app.name}': {version}..."

        spinner.enabled = True
        app.install(
            version=version,
            install_dependencies=True,
            python_bin=sys.executable,
            stdout=logs,
        )
except Exception as error:
    header.value = f"Error while trying to install the app: {error}"
else:
    redirect = query.get("redirect", None)
    if redirect:
        redirect_url = urlparse.urlunsplit(
            url._replace(path=redirect[0], query=None, fragment=None)
        )
        header.value = (
            f'Redirecting now to <a href="{redirect_url}">{redirect_url}</a>...'
        )
        js = Javascript(f"window.location = '{redirect_url}'")
        display(js)
finally:
    spinner.enabled = False